## Keras tuner :- decides number of hidden layers and neurons in neural network

In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

<ipython-input-1-63cf4df417c7>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
df=pd.read_csv('Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [3]:
X=df.iloc[:,:-1]
y=df.iloc[:,1]

In [4]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        #here hp is parameter when we are are running randomsearch and we are saying neural network can have layers from 2 to 20 (hidden layers)
        model.add(layers.Dense(units=hp.Int('unit_'+ str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
        #number of neurons ranges from 32 to 512
    model.add(layers.Dense(1,activation='linear'))
    #since this is regression problem so output will be only 1
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',[.001,.0001,.00001])),
            loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model                  
        

In [9]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='deep_learning')

In [10]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
unit_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
unit_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 1e-05], 'ordered': True}


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [12]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 19s]
val_mean_absolute_error: 5.523187478383382

Best val_mean_absolute_error So Far: 3.081889549891154
Total elapsed time: 00h 00m 59s
INFO:tensorflow:Oracle triggered exit


In [13]:
tuner.results_summary()

Results summary
Results in deep_learning\untitled_project
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 6
unit_0: 192
unit_1: 160
learning_rate: 0.001
unit_2: 64
unit_3: 256
unit_4: 64
unit_5: 448
unit_6: 384
unit_7: 224
unit_8: 160
unit_9: 32
unit_10: 256
unit_11: 192
unit_12: 224
unit_13: 160
unit_14: 64
unit_15: 256
unit_16: 224
unit_17: 416
unit_18: 384
unit_19: 384
Score: 3.081889549891154
Trial summary
Hyperparameters:
num_layers: 4
unit_0: 448
unit_1: 96
learning_rate: 0.0001
unit_2: 256
unit_3: 480
unit_4: 128
unit_5: 64
unit_6: 288
unit_7: 288
unit_8: 288
unit_9: 416
unit_10: 384
unit_11: 320
unit_12: 64
unit_13: 416
unit_14: 352
unit_15: 64
unit_16: 160
unit_17: 320
unit_18: 320
unit_19: 384
Score: 3.6340412298838296
Trial summary
Hyperparameters:
num_layers: 18
unit_0: 160
unit_1: 224
learning_rate: 0.0001
unit_2: 96
unit_3: 352
unit_4: 320
unit_5: 128
unit_6: 128
unit_7: 288
unit_8: 480
unit_9: 2

In [ ]:
# therefore we can choose and create the neural network which performs best
